# LLM agent Hackathon 

## Alireza Ghafarollahi, MIT

# Create your first agent
- ## LLM: GPT-4o
- ## Task: write a Python code


In [13]:
import json
import os
import re

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [14]:
from openai import OpenAI
import json
import os.path as osp
import subprocess

client = OpenAI(organization ='')

In [49]:
def extract_json_between_markers(llm_output):
    # Regular expression pattern to find JSON content between ```json and ```
    json_pattern = r"```json(.*?)```"
    matches = re.findall(json_pattern, llm_output, re.DOTALL)

    if not matches:
        # Fallback: Try to find any JSON-like content in the output
        json_pattern = r"\{.*?\}"
        matches = re.findall(json_pattern, llm_output, re.DOTALL)

    for json_string in matches:
        json_string = json_string.strip()
        try:
            parsed_json = json.loads(json_string)
            return parsed_json
        except json.JSONDecodeError:
            # Attempt to fix common JSON issues
            try:
                # Remove invalid control characters
                json_string_clean = re.sub(r"[\x00-\x1F\x7F]", "", json_string)
                parsed_json = json.loads(json_string_clean)
                return parsed_json
            except json.JSONDecodeError:
                continue  # Try next match

    return None  # No valid JSON found
    
def get_response_from_llm(
        system_message,
        prompt,
        model,
        reasoning_effort="medium",
        print_debug=False,
        msg_history=None,
        temperature=0.75,
        client=client):

    if msg_history is None:
        msg_history = []

    new_msg_history = msg_history + [{"role": "user", "content": prompt}]
    
    if model in ["gpt-4o", "gpt-4-turbo"]:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "developer", "content": system_message},
                *new_msg_history,
            ],
            temperature=temperature,
            max_completion_tokens=15000
        )
        print(token_usage(response))

    elif model in ["gpt-4.1"]:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "developer", "content": system_message},
                *new_msg_history,
            ],
            temperature=temperature,
            max_completion_tokens=20000
        )
        print(token_usage(response))
        
    elif model in ["o1", "o1-mini", "o3", "o3-mini"]:
        response = client.chat.completions.create(
            model=model,
            reasoning_effort=reasoning_effort,
            messages=[
                {"role": "developer", "content": system_message},
                *new_msg_history,
            ],
        )
        print(token_usage(response))
        
    content = response.choices[0].message.content
    new_msg_history = new_msg_history + [{"role": "assistant", "content": content}]

    if print_debug:
        print()
        print("*" * 20 + " LLM START " + "*" * 20)
        for j, msg in enumerate(new_msg_history):
            print(f'{j}, {msg["role"]}: {msg["content"]}')
            print("*" * 51)
        #print(content)
        print("*" * 21 + " LLM END " + "*" * 21)
        print()

    return content, new_msg_history

def token_usage(response):
    dic = {"prompt tokens": response.usage.prompt_tokens,
          "completion tokens" : response.usage.completion_tokens,
          "total tokens": response.usage.total_tokens,
           "reasoning tokens": response.usage.completion_tokens_details.reasoning_tokens
          }
    return dic

In [50]:
system_message = '''You are a senior coder with expertise in writing Python codes. 
'''

prompt = ''' You carefully write a Python code for the given task.

INSTRUCTION:
- Always save the plots, rather than showing them.

Respond in the following format

<code_START>
<CODE>
<code_FINISH>

Replace <CODE> with your complete and well-organized Python script. 

Additional note:
Write just the raw Python logic, no Markdown formatting, and no main() function wrapper, no ```python in the beginning and ``` in the end.

TASK:
{task}
'''

task = '''Derive the mathematical expression for the roots of a third degree polynomial. 
Then give a sample function and compute its roots using the expressions. Finally, plot the function, showing the roots and annotaing the mathematical expression.''' 

code_hosory = []

code, code_history = get_response_from_llm(system_message=system_message,
                                          model='gpt-4o',
                                          prompt=prompt.format(task=task),
                                          temperature=0,
                                          print_debug=True)

{'prompt tokens': 162, 'completion tokens': 439, 'total tokens': 601, 'reasoning tokens': 0}

******************** LLM START ********************
0, user:  You carefully write a Python code for the given task.

INSTRUCTION:
- Always save the plots, rather than showing them.

Respond in the following format

<code_START>
<CODE>
<code_FINISH>

Replace <CODE> with your complete and well-organized Python script. 

Additional note:
Write just the raw Python logic, no Markdown formatting, and no main() function wrapper, no ```python in the beginning and ``` in the end.

TASK:
Derive the mathematical expression for the roots of a third degree polynomial. 
Then give a sample function and compute its roots using the expressions. Finally, plot the function, showing the roots and annotaing the mathematical expression.

***************************************************
1, assistant: <code_START>
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, solve, Eq, latex

# Def

In [51]:
match = re.search(r'<code_START>(.*?)<code_FINISH>', code, re.DOTALL)
if match:
    code_block = match.group(1).strip()
else:
    print("Code block not found.")
    
with open(f'python_code.py', 'w') as f:
    f.writelines(code_block)
        
command = ['python', f"python_code.py"]            
    
result = subprocess.run(command, stderr=subprocess.PIPE, text=True)

In [52]:
print(result)

CompletedProcess(args=['python', 'python_code.py'], returncode=0, stderr="findfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXGeneral'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXNonUnicode'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXSizeOneSym'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXSizeTwoSym'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXSizeThreeSym'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXSizeFourSym'] not found. Falling back to DejaVu Sans.\nfindfont: Font family ['STIXSizeF